In [1]:
import numpy as np
import pandas as pd
import torch
import pickle
from Functions_generation import generate_a_song_structure, sample_with_temp_topk, CharRNN

In [2]:
with open(r"..\Corpus\Encoding_RNN_LSTM\Char_level\encoding_map.pkl", "rb") as f:
    mapping = pickle.load(f)

int2char = {i: ch for ch, i in mapping.items()}
nb_char = len(int2char)

In [3]:
matrix = pd.read_csv(r"..\Markov\transition_matrix.csv")

states = np.array(matrix.iloc[6])
prob_transi = np.array(matrix.iloc[0:6])

In [5]:
embedding_dim = 64
hidden_size = 512
vocab_size = len(mapping)

model = CharRNN(vocab_size, embedding_dim, hidden_size, num_layers=2)   

# Teacher forcing = 1 during training

In [10]:
ckpt = torch.load("..\Models\RNN_model_full_teacher_forced.pt", map_location="cpu")

new_state_dict = {}
for k, v in ckpt["model_state_dict"].items():
    new_key = k.replace("_orig_mod.", "")  # supprime le préfixe "origin."
    new_state_dict[new_key] = v

In [11]:
model.load_state_dict(new_state_dict)

<All keys matched successfully>

I will give the same structure for RNN and LSTM to compare

In [12]:
struct = generate_a_song_structure(prob_transi.astype(float),states)

#struct = ['<BEGINNING>', '<COUPLET>', '<REFRAIN>', '<COUPLET>', '<REFRAIN>', '<END>']
struct = ['α', 'γ', 'ε', 'γ', 'ε', 'θ']

encoded = torch.tensor([mapping[c] for c in struct[1]], dtype=torch.long).unsqueeze(0)

['<BEGINNING>', '<COUPLET>', '<COUPLET>', '<REFRAIN>', '<OUTRO>', '<END>']


Forbidden to generate

In [13]:
forbidden_generation = [i for i in range(64,72)]
[int2char[i] for i in forbidden_generation]

['α', 'β', 'γ', 'ε', 'ζ', 'η', 'θ', '€']

In [14]:
decod_structure = {"β" : "<INTRO>",
                   "γ" : "<COUPLET>",
                   "ε" : "<REFRAIN>",
                   "ζ" : "<PONT>",
                   "η" : "<OUTRO>",
                   "θ" : "<END>"}

In [15]:
context_gen = "voila pourquoi"
context_encode = [mapping[i] for i in context_gen]
context_size = len(context_encode)

hid = model.init_hidden(batch_size=1)

for i in range(len(struct) - 2):

    print()
    print(decod_structure[struct[i+1]])

    if i == 0 : 
        context_encode.insert(0,mapping[struct[i+1]])
        context_encode.insert(1,mapping["\n"])
        encoded = torch.tensor(context_encode, dtype=torch.long).unsqueeze(0)

        print("".join([int2char[i] for i in context_encode[1:]]), end="")

        out, hid = model(encoded, hid)
        next_input = encoded[:, -1].unsqueeze(0)  # shape (1, 1)

    else : 
        #next_input = torch.tensor(mapping[struct[i+1]],dtype=torch.long).unsqueeze(0)
        next_input = torch.tensor(mapping[struct[i+1]],dtype=torch.long).view(1,1)

    last_input = 0
    first_input = True

    while last_input != 12:
        with torch.no_grad():
            out, hid = model(next_input, hid)   # out shape: (1, 1, vocab_size)

            out[0, -1][forbidden_generation] = float("-inf")

        # 4. On échantillonne un token
            next_token = sample_with_temp_topk(out[0, -1], temperature=0.5, top_k=20)
            last_input = next_token.item()

        # 5. Conditions d’impression
            if first_input and last_input == 12:  # Si début et le modèle sort directement <END>
                last_input = 0
                continue

            elif first_input and last_input == 0:  # Si début et sortie = \n
                first_input = False
                forbidden_generation.extend([0])
                print(int2char[next_token.item()], end="")

            elif next_token.item() != 12:
                first_input = False
                print(int2char[next_token.item()], end="")

                if 0 in forbidden_generation:
                    forbidden_generation.pop(-1)

        # 6. Préparer la prochaine entrée
            next_input = next_token.view(1, 1)

print("\n", decod_structure[struct[-1]])



<COUPLET>

voila pourquoij'suis pas d'la couvre
les bras
et moi j'ai pas d'habitude, tu m'verras plus en vrai et c'est réel, j'suis avec des mots d'la présence de porte de la merde
alors j'ai fait du gardien sur le biff à ses potes sont pas d'armes
j'ai pas d'quartier d'mes toxes
si l'amitié dans la bouche te parle de ma perte de la bouche
quand les mecs, j'ai plus d'amis au peu d'fric au moins qu'il faut qu'on fout d'l'arme de gars à la routine avec vos mots sont pas prêts à la poster le diable qui m'a ramené des malheurs d'un peu d'autre comme si j'étais le compte en assurer
j'ai l'impression qu'on s'appelle dans la rue prémiser les portes de nous c'est l'barillet, j'ai des papiers
on sait qu'on t'a dit qu'j'ai peur de l'amour et d'ma vie fait des rappeurs qui vont l'apprécier les mauvaises contrôlent au visage
j'ai dit qu't'es pas devenir triste fois des choses qui restent de l'amour, j'suis là où j'ai l'air du bonheur qui s'désolé en espérant qu'elle est dehors de la série d'la me

# If no restriction on the generation part 

In [16]:
context_gen = "malice"
context_encode = [mapping[i] for i in context_gen]

hid = model.init_hidden(batch_size=1)

encoded = torch.tensor(context_encode, dtype=torch.long).unsqueeze(0)
print("".join([int2char[i] for i in context_encode]), end="")

out, hid = model(encoded, hid)
next_input = encoded[:, -1].unsqueeze(0) 

last_input = 0
first_input = True

while last_input != 70:
    with torch.no_grad():
        out, hid = model(next_input, hid) 

        next_token = sample_with_temp_topk(out[0, -1], temperature=0.5, top_k=20)

        if (last_input == 0) and (next_token != list(decod_structure.keys())):
            print(int2char[next_token.item()].upper(), end="")
        elif last_input != 70 : 
            print(int2char[next_token.item()], end="")

        last_input = next_token.item()
        next_input = next_token.view(1, 1)

malice
J'ride à ces fautes de merde j'ai pas envie d'être là où j'suis rentrer tout le monde changer d'la police sans la fin d'mon pote c'est mieux qu'on découpe l'avenir qu'on s'embrase de la rue par la finesse, j'suis mentir les sons de l'argent de pas de garde des textes qu'on va tout c'que j'ai pas de partie d'l'amour est comme le centre de la peur
Le coup d'vin du matin loin d'un coup d'malchance de l'ombre et des coups d'couilles, des points d'choses qui vont pas faire du sang de la route est pas comme si j'étais la soulever les mains au préfire pour les mains s'entretrouvent pas d'putes, je m'en rappelle qu'on t'en sorte de brisé d'mes binoles comme les putes de toute la mort j'ai des p'tites sentiments d'paname d'amis, on a envie d'me dire qu'les phases se calculent pas les portes et les mains comme si j'avais pas l'temps d'men chez moi, j'suis dans l'cercle
J'suis bien tout près du barillet et les p'tits qui partent pas d'shit
J'ai déjà d'jà mieux qu'c'est pour mes mauvais cha

# Teacher forcing < 1 toward the end of training

In [17]:
ckpt = torch.load("..\Models\RNN_model_not_full_forced.pt", map_location="cpu")
new_state_dict = {}
for k, v in ckpt["model_state_dict"].items():
    new_key = k.replace("_orig_mod.", "")  # supprime le préfixe "origin."
    new_state_dict[new_key] = v

In [18]:
#ckpt = torch.load("..\Models\LSTM_model_not_full_forced.pt", map_location="cpu")
#torch.compile(model)
model.load_state_dict(new_state_dict)

<All keys matched successfully>

Forbidden to generate

In [20]:
context_gen = "voila pourquoi"
context_encode = [mapping[i] for i in context_gen]
context_size = len(context_encode)

hid = model.init_hidden(batch_size=1)

for i in range(len(struct) - 2):

    print()
    print(decod_structure[struct[i+1]])

    if i == 0 : 
        context_encode.insert(0,mapping[struct[i+1]])
        context_encode.insert(1,mapping["\n"])
        encoded = torch.tensor(context_encode, dtype=torch.long).unsqueeze(0)

        print("".join([int2char[i] for i in context_encode[1:]]), end="")

        out, hid = model(encoded, hid)
        next_input = encoded[:, -1].unsqueeze(0)  # shape (1, 1)

    else : 
        #next_input = torch.tensor(mapping[struct[i+1]],dtype=torch.long).unsqueeze(0)
        next_input = torch.tensor(mapping[struct[i+1]],dtype=torch.long).view(1,1)

    last_input = 0
    first_input = True

    while last_input != 12:
        with torch.no_grad():
            out, hid = model(next_input, hid)   # out shape: (1, 1, vocab_size)

            out[0, -1][forbidden_generation] = float("-inf")

        # 4. On échantillonne un token
            next_token = sample_with_temp_topk(out[0, -1], temperature=0.5, top_k=20)
            last_input = next_token.item()

        # 5. Conditions d’impression
            if first_input and last_input == 12:  # Si début et le modèle sort directement <END>
                last_input = 0
                continue

            elif first_input and last_input == 0:  # Si début et sortie = \n
                first_input = False
                forbidden_generation.extend([0])
                print(int2char[next_token.item()], end="")

            elif next_token.item() != 12:
                first_input = False
                print(int2char[next_token.item()], end="")

                if 0 in forbidden_generation:
                    forbidden_generation.pop(-1)

        # 6. Préparer la prochaine entrée
            next_input = next_token.view(1, 1)

print("\n", decod_structure[struct[-1]])



<COUPLET>

voila pourquoitu sais pas les states de toutes les deux pas mes deux bougies sont tout mal de souvenir  lls me pas de paper  plus de tense s'il fait les nouvelles font peur, c'est pas les coursss pour le  fleurs et les blagues de son doigts de ces tours et j'fais pas les mains sont toutes les montres, sage dans les plus de bars   deux points de ta merde
j'me dis qu'j'ai pas d'regard de mon pote dans la maille

<REFRAIN>

j'suis dans la mort de mes plattes les mots ee monde de mes regrets sont faut mais j'me rappelle toutes les bougies de la maison est un son de porsse, vers les deux sont les problèmes sont les bougies moiss de mon coeur de mon contraire, j'ai tout peur de les scheets
le  coups de sous  mama les yeux mais j'écoute pas de  plus forts de la force de saille , tu m'les vois pas les plus coups d'prrents de courte sont des polites de tout les plus trop de mes soumis comme un petit décider
c'est moi qui devient un faux comme l'école
j'suis un  femme d'aart mon rap,

# If no restriction on the generation part 

In [21]:
context_gen = "malice"
context_encode = [mapping[i] for i in context_gen]

hid = model.init_hidden(batch_size=1)

encoded = torch.tensor(context_encode, dtype=torch.long).unsqueeze(0)
print("".join([int2char[i] for i in context_encode]), end="")

out, hid = model(encoded, hid)
next_input = encoded[:, -1].unsqueeze(0) 

last_input = 0
first_input = True

while last_input != 70:
    with torch.no_grad():
        out, hid = model(next_input, hid) 

        next_token = sample_with_temp_topk(out[0, -1], temperature=0.5, top_k=20)

        if (last_input == 0) and (next_token != list(decod_structure.keys())):
            print(int2char[next_token.item()].upper(), end="")
        elif last_input != 70 : 
            print(int2char[next_token.item()], end="")

        last_input = next_token.item()
        next_input = next_token.view(1, 1)

malice
Donc tu veux pas me faire mon école
Toutes les jours
J'suis dans les meufs m'appellent des pooblèmes de con  mais j'écoute des couplets les blessures s'éteignent la soie de l'argent par la cause de belgique
On est comme si j'étais comme les couplets sont foutures
Les sous de mon plus bien de salale  mais on reste aas si si tes potes d'homme pas d'bruit d'la soupe de rêve d'un plug, j'ai bien quand j'mettais pas les battes sont dans l'fond
J'ai pas b'soin d'causer des fleurs font toutes les couplets d'plus belle la peur d'la musique
J'ai un poaier sur le  chauds
J'suis pas là pour les courtes
J'ai pas d'la seule plume de prod  comme ma maison, j'te dis le  vegresses de la plage est trop là, j'ai mon coup d'contrôle, c'est pas mon aroupe des blessures de la vitesse de mon partinique, t'as des renses, j'me rends l'disciple
Quelques pattes  j'ai peur de moi comme un pote vie d'un peu de paix
/γ
Θ

In [3]:
matrix = pd.read_csv(r"..\Markov\transition_matrix.csv")

states = np.array(matrix.iloc[6])
prob_transi = np.array(matrix.iloc[0:6])

# Model loading

In [4]:
vocab_size = len(int2char)
hidden_size = 250

model = CharRNN(vocab_size, hidden_size, num_layers=3)   

ckpt = torch.load("..\Models\RNN_model.pt", map_location="cpu")
model.load_state_dict(ckpt["model_state_dict"])

<All keys matched successfully>

The char token representing the end of a section is at place 79

In [5]:
struct = generate_a_song_structure(prob_transi.astype(float),states)

encoded = torch.tensor([mapping[c] for c in struct[1]], dtype=torch.long).unsqueeze(0)

['<BEGINNING>', '<INTRO>', '<COUPLET>', '<OUTRO>', '<END>']


In [6]:
hid = model.init_hidden(batch_size=1)

for i in range(len(struct)-2) :

    print()
    print(struct[i+1])
    encoded = torch.tensor([mapping[c] for c in struct[i+1]], dtype=torch.long).unsqueeze(0)

    # prime the model with your encoded context
    out, hid = model(encoded, hid)

    # take the last token of the context
    next_input = encoded[:, -1].unsqueeze(0)  # shape (1, 1)

    last_input = 0
    first_input = True

    while last_input != 79 :
        with torch.no_grad():
            out, hid = model(next_input, hid)   # out shape: (1, 1, vocab_size)

    # sample from logits
        next_token = sample_with_temp_topk(out[0, -1], temperature=0.5, top_k=20)
        last_input = next_token.item()

    # print char
        if first_input and last_input == 79 :
            last_input = 0
            continue

        if next_token.item() != 79 :
            first_input = False
            print(int2char[next_token.item()], end="")


    # prepare next input (still (1, 1))
        next_input = next_token.view(1, 1)

print("\n",struct[-1])


<INTRO>

J'ai trop de rap de part les cames
Plus de la vie de Port de la main de la cerveau nom du Havre
Le cours
J'ai mis le porter le monde sort de passer le bout de la ville de respectif
Le monde qu'à la pousse de l'homme de barbe de courrer les écoute le lastrune
La ville dans le mature en mon petite mort se parle de la foule mais un flow de leur fusil d'accusé de La plume de l'argent de la fin du Homme avec le salaire comme les petours de la plume de suiver au contier quand les basses de mal comme des fois qui s'monte dans les masses de la faime du bruit de l'arration
Comme le mandant des gros cartes de la rue
Dans la puissance du Havrais de l'espectin de la pression de son habille dans les bouts de la musicule de l'ais de la main de crocation de faire la pertisse de mon sport sont des mosses année du tiers d'un contre une faire les fonds par la plage de l'attertaine
J'ai fais mieux qu'on en apres de mes plaises mais le garde au fond de ceux qui parle des armes de la rue tout le 